#Loading and organizing the data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
!pip install rarfile
import rarfile
import os
import random
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from shutil import copyfile

     |████████████████████████████████| 122kB 2.7MB/s 
  Created wheel for rarfile: filename=rarfile-3.1-cp36-none-any.whl size=24908 sha256=9ebbbe6988f3c6fb40ff97da90c1aa8d63d11452a8762289a1561756add07e73
  Stored in directory: /root/.cache/pip/wheels/23/3c/c8/0215b6a5079492eff3be3f545ae0b0c4a66734c35c9e444eac
Successfully built rarfile


In [ ]:
#extracting the data

def unrar(source_path,dest_path):
  for rar in os.listdir(source_path):
    filepath = os.path.join(source_path, rar)
    with rarfile.RarFile(filepath) as opened_rar:
      #for f in opened_rar.infolist():
        #print (f.filename, f.file_size)
      opened_rar.extractall(os.path.join(dest_path))

unrar("/content/drive/My Drive/Kaggle - COVID-19 and Pneumonia Diagnosis/","/content/drive/My Drive/Kaggle - COVID-19 and Pneumonia Diagnosis/extracted")

In [ ]:
# Making sure data was successfully extracted
print(len(os.listdir('/content/drive/My Drive/Kaggle - COVID-19 and Pneumonia Diagnosis/extracted/train/COVID19 AND PNEUMONIA/')))
print(len(os.listdir('/content/drive/My Drive/Kaggle - COVID-19 and Pneumonia Diagnosis/extracted/train/NORMAL/')))
print(len(os.listdir('/content/drive/My Drive/Kaggle - COVID-19 and Pneumonia Diagnosis/extracted/test/')))

3925
1341
690


In [ ]:
# Automated Keras Train/Validation Split

# train_datagen = ImageDataGenerator(rescale=1./255,
#     shear_range=0.2,
#     zoom_range=0.2,
#     horizontal_flip=True,
#     validation_split=0.1) # set validation split

# train_generator = train_datagen.flow_from_directory(
#     train_data_dir,
#     target_size=(img_height, img_width),
#     batch_size=batch_size,
#     class_mode='binary',
#     subset='training') # set as training data

# validation_generator = train_datagen.flow_from_directory(
#     train_data_dir, # same directory as training data
#     target_size=(img_height, img_width),
#     batch_size=batch_size,
#     class_mode='binary',
#     subset='validation') # set as validation data

In [ ]:
os.mkdir('/tmp/Kaggle - COVID-19 and Pneumonia Diagnosis')
os.mkdir('/tmp/Kaggle - COVID-19 and Pneumonia Diagnosis/training')
os.mkdir('/tmp/Kaggle - COVID-19 and Pneumonia Diagnosis/testing')
os.mkdir('/tmp/Kaggle - COVID-19 and Pneumonia Diagnosis/training/covid')
os.mkdir('/tmp/Kaggle - COVID-19 and Pneumonia Diagnosis/training/normal')
os.mkdir('/tmp/Kaggle - COVID-19 and Pneumonia Diagnosis/testing/covid')
os.mkdir('/tmp/Kaggle - COVID-19 and Pneumonia Diagnosis/testing/normal')

In [ ]:
def split_data(SOURCE, TRAINING, TESTING, SPLIT_SIZE):
    files = []
    for filename in os.listdir(SOURCE):
        file = SOURCE + filename
        files.append(filename)

    training_length = int(len(files) * SPLIT_SIZE)
    testing_length = int(len(files) - training_length)
    shuffled_set = random.sample(files, len(files))
    training_set = shuffled_set[0:training_length]
    testing_set = shuffled_set[:testing_length]

    for filename in training_set:
        this_file = SOURCE + filename
        destination = TRAINING + filename
        copyfile(this_file, destination)

    for filename in testing_set:
        this_file = SOURCE + filename
        destination = TESTING + filename
        copyfile(this_file, destination)


COVID_SOURCE = "/content/drive/My Drive/Kaggle - COVID-19 and Pneumonia Diagnosis/extracted/train/COVID19 AND PNEUMONIA/"
TRAINING_COVID_DEST = "/tmp/Kaggle - COVID-19 and Pneumonia Diagnosis/training/covid/"
TESTING_COVID_DEST = "/tmp/Kaggle - COVID-19 and Pneumonia Diagnosis/testing/covid/"
NORMAL_SOURCE = "/content/drive/My Drive/Kaggle - COVID-19 and Pneumonia Diagnosis/extracted/train/NORMAL/"
TRAINING_NORMAL_DEST = "/tmp/Kaggle - COVID-19 and Pneumonia Diagnosis/training/normal/"
TESTING_NORMAL_DEST = "/tmp/Kaggle - COVID-19 and Pneumonia Diagnosis/testing/normal/"

split_size = .9
split_data(COVID_SOURCE, TRAINING_COVID_DEST, TESTING_COVID_DEST, split_size)
split_data(NORMAL_SOURCE, TRAINING_NORMAL_DEST, TESTING_NORMAL_DEST, split_size)


In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer=RMSprop(lr=0.001), loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
TRAINING_DIR = "/tmp/Kaggle - COVID-19 and Pneumonia Diagnosis/training/"
train_datagen = ImageDataGenerator(rescale=1./255,
      #rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      #horizontal_flip=True,
      fill_mode='nearest',
      bal)
train_generator = train_datagen.flow_from_directory(TRAINING_DIR,
                                                    batch_size=20,
                                                    class_mode='binary',
                                                    target_size=(150, 150))

VALIDATION_DIR = "/tmp/Kaggle - COVID-19 and Pneumonia Diagnosis/testing/"
validation_datagen = ImageDataGenerator(rescale=1./255
    #   rotation_range=40,
    #   width_shift_range=0.2,
    #   height_shift_range=0.2,
    #   shear_range=0.2,
    #   zoom_range=0.2,
    #   #horizontal_flip=True,
    #   fill_mode='nearest')
validation_generator = validation_datagen.flow_from_directory(VALIDATION_DIR,
                                                              batch_size=20,
                                                              class_mode='binary',
                                                              target_size=(150, 150))

Found 4738 images belonging to 2 classes.
Found 528 images belonging to 2 classes.


In [ ]:
m = model.fit(train_generator,
              epochs=15,
              steps_per_epoch= 4738/20,
              validation_steps = 50,
              verbose=1,
              validation_data=validation_generator)

Epoch 1/15
237/236 [==============================] - 77s 323ms/step - loss: 0.3444 - accuracy: 0.8506 - val_loss: 0.4086 - val_accuracy: 0.8248
Epoch 2/15
237/236 [==============================] - 77s 327ms/step - loss: 0.2786 - accuracy: 0.8805 - val_loss: 0.3531 - val_accuracy: 0.8563
Epoch 3/15
237/236 [==============================] - 77s 326ms/step - loss: 0.2538 - accuracy: 0.8964 - val_loss: 0.3392 - val_accuracy: 0.8361
Epoch 4/15
237/236 [==============================] - 77s 326ms/step - loss: 0.2343 - accuracy: 0.9025 - val_loss: 0.2848 - val_accuracy: 0.8770
Epoch 5/15
237/236 [==============================] - 77s 325ms/step - loss: 0.2237 - accuracy: 0.9171 - val_loss: 1.0390 - val_accuracy: 0.6455
Epoch 6/15
237/236 [==============================] - 77s 325ms/step - loss: 0.2111 - accuracy: 0.9215 - val_loss: 0.2686 - val_accuracy: 0.8781
Epoch 7/15
237/236 [==============================] - 77s 324ms/step - loss: 0.1952 - accuracy: 0.9276 - val_loss: 0.3431 - val_ac

In [ ]:
files = os.listdir('/content/drive/My Drive/Kaggle - COVID-19 and Pneumonia Diagnosis/extracted/test/')
import shutil

for f in files: 
    fi = os.path.join('/content/drive/My Drive/Kaggle - COVID-19 and Pneumonia Diagnosis/extracted/test/', f)
    shutil.move(fi, '/content/drive/My Drive/Kaggle - COVID-19 and Pneumonia Diagnosis/extracted/test/images')

In [ ]:
#train_generator.class_indices
import pandas as pd
#sub = pd.read_csv('/content/drive/My Drive/Kaggle - COVID-19 and Pneumonia Diagnosis/Submit.csv')
pred_dir = '/content/drive/My Drive/Kaggle - COVID-19 and Pneumonia Diagnosis/extracted/test/'
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    directory=pred_dir,
    target_size=(150, 150),
    batch_size=1,
    #class_mode='binary',
    shuffle=False
)

Found 690 images belonging to 1 classes.


In [ ]:
#test_generator.reset()
pred=model.predict_generator(test_generator, steps=len(test_generator), verbose=1)
predicted_class_indices=np.argmax(pred,axis=1)
cl = np.round(pred)

690/690 [==============================] - 9s 13ms/step


In [ ]:
filenames=[i[7:] for i in test_generator.filenames]

#print(cl)
classes = [int(i) for i in cl]
print(classes)
for i in range(len(classes)):
  if classes[i] == 1:
    classes[i] = 0
  else:
    classes[i] = 1
results=pd.DataFrame({"Image":filenames,
                      "Label":classes})
print(results)
results.to_csv('Submit.csv', index=False)

[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [ ]:
import os, shutil
shutil.rmtree('/content/drive/My Drive/Kaggle - COVID-19 and Pneumonia Diagnosis/extractedBackup/test')
shutil.rmtree('/content/drive/My Drive/Kaggle - COVID-19 and Pneumonia Diagnosis/extractedBackup/train')

In [ ]:
def copytree(src, dst, symlinks=False, ignore=None):
    for item in os.listdir(src):
        s = os.path.join(src, item)
        d = os.path.join(dst, item)
        if os.path.isdir(s):
            shutil.copytree(s, d, symlinks, ignore)
        else:
            shutil.copy2(s, d)


fromDirectory = "/content/drive/My Drive/Kaggle - COVID-19 and Pneumonia Diagnosis/extracted/"
toDirectory = "/content/drive/My Drive/Kaggle - COVID-19 and Pneumonia Diagnosis/extractedBackup/"

copytree(fromDirectory, toDirectory)


In [ ]:
import os, shutil

files = os.listdir('/content/drive/My Drive/Kaggle - COVID-19 and Pneumonia Diagnosis/extracted/train/NORMAL/NORMAL/')

#os.mkdir('/content/drive/My Drive/Kaggle - COVID-19 and Pneumonia Diagnosis/extracted/train/NORMAL/NORMAL')

for f in files: 
    fi = os.path.join('/content/drive/My Drive/Kaggle - COVID-19 and Pneumonia Diagnosis/extracted/train/NORMAL/NORMAL/', f)
    shutil.move(fi, '/content/drive/My Drive/Kaggle - COVID-19 and Pneumonia Diagnosis/extracted/train/NORMAL/')

In [ ]:
shutil.rmtree('/content/drive/My Drive/Kaggle - COVID-19 and Pneumonia Diagnosis/extracted/train/NORMAL/NORMAL/')

In [ ]:
import cv2
from skimage import exposure
import numpy as np

raw_files = os.listdir('/content/drive/My Drive/Kaggle - COVID-19 and Pneumonia Diagnosis/extracted/train/NORMAL/')

for f in raw_files:
    # if "Augmented" in f:
    #     os.remove(os.path.join('/content/drive/My Drive/Kaggle - COVID-19 and Pneumonia Diagnosis/extracted/train/NORMAL/', f))
    raw_img = cv2.imread(os.path.join('/content/drive/My Drive/Kaggle - COVID-19 and Pneumonia Diagnosis/extracted/train/NORMAL/', f))
    if raw_img is not None:
        aug_img1 = cv2.flip(raw_img, 1)

        v_min, v_max = np.percentile(raw_img, (0.2, 99.8))
        aug_img2 = exposure.rescale_intensity(raw_img, in_range=(v_min, v_max))

        aug_name1 = os.path.join('/content/drive/My Drive/Kaggle - COVID-19 and Pneumonia Diagnosis/extracted/train/NORMAL/', '(Augmented1)'+ f +'.png')
        aug_name2 = os.path.join('/content/drive/My Drive/Kaggle - COVID-19 and Pneumonia Diagnosis/extracted/train/NORMAL/', '(Augmented2)'+ f +'.png')
        cv2.imwrite(aug_name1, aug_img1)
        cv2.imwrite(aug_name2, aug_img2)
    

In [ ]:
print(len(os.listdir('/content/drive/My Drive/Kaggle - COVID-19 and Pneumonia Diagnosis/extracted/train/NORMAL')))

4024
